In [76]:
#%%writefile igs_v1.py

# It has start token and no masking words excluded

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from matplotlib.colors import Normalize, rgb2hex
import matplotlib as mpl
from IPython.display import HTML
import tensorflow_text as text
import tensorflow_datasets as tfds


class IntegratedGradients:
    def __init__(self, model, vectorization_layer_index, embedding_layer_index, vocab_=None, display=True):
        
        ## Set display model
        self.display = display
        
        #language model        
        self.model = model
        
        #Get embedding layer
        self.embed_layer = self.model.layers[embedding_layer_index]
        
        # build new model with all layers after embedding layer
        self.new_model = tf.keras.Sequential()
        for layer in self.model.layers[embedding_layer_index+1:]:
            #print(layer.name)
            self.new_model.add(layer)
            
        ## Get the vectorization layer
        self.encoder = self.model.layers[vectorization_layer_index]
        
        ##get the vocabulary
        if vocab_ is None:
            self.vocab = self.encoder.get_vocabulary()
        else:
            self.vocab = vocab_
        
    def interpret(self,sample_sentence, target_label_index, actual_label, n_steps=50):
        """
        Returns the interpretations in HTML format word attribution plots
        Input Arguments
        
            sample_sentence: input sentence for which prediction and explanation required for
            target_label_index: If binary problem always 0, for sotmax output layer models, the actual prediciton index.
            n_steps: number of interpolation steps between baseline input and actual input, actual paper suggests, anything above 30
        
        """
        
        # Get the vector representation for the given sentence
        #sample_vector = self.encoder([sample_sentence])
        sample_vector = np.squeeze(self.encoder([sample_sentence]).numpy()).tolist()

        # words of the given vector as per trained vocabulary, used later for plotting
        words = [self.vocab[i] for i in sample_vector]    

        # get embeddings
        encoded = self.encoder([sample_sentence])
        sample_embed = self.embed_layer(encoded)
        #print(f"Shape of the sample_embed: {sample_embed.shape}")
        #return

        # Create a Baseline vector with zero embeddings
        baseline_embed = tf.zeros(shape=tf.shape(sample_embed))

        # Interpolate texts between base line sentence and the actual sentense
        interpolated_texts = self.interpolate_texts(baseline_embed,
                                           sample_embed,
                                           n_steps)
        #return interpolated_texts, sample_vector_mask
        # Make the padded words to zere based on mask
        #embed_mask = tf.stack([tf.stack([tf.squeeze(sample_vector_mask,axis=0)]*interpolated_texts.shape[0])]*interpolated_texts.shape[-1],axis=-1)
        #interpolated_texts = tf.math.multiply(interpolated_texts,tf.cast(embed_mask,tf.float32))
        

        # Compute gradients for the interploated texts    
        path_gradients = self.compute_gradients(interpolated_texts, target_label_index)
        #print(f"Shape of the path_gradients: {path_gradients.shape}")

        # sum the grads of the interpolated vectors
        all_grads = tf.reduce_sum(path_gradients, axis=0) / n_steps
        #print(f"Shape of all_grads: {all_grads.shape}")
        # mulitply grads by (input - baseline); baseline is zero vectors
        x_grads = tf.math.multiply(all_grads, sample_embed)
        #print(f"Shape of x_grads: {x_grads.shape}")
        # sum all gradients across the embedding dimension
        igs = tf.reduce_sum(x_grads, axis=-1).numpy()
        igs = tf.squeeze(igs)
        #print(f"Shape of igs: {igs.shape}")
        
        #Filter IGs for only valid words
        #pad_start = tf.argmin(encoded, axis=1).numpy()[0]
        #print(f"Pad start: {pad_start}")
        #igs = igs[1:pad_start]
        #words = words[1:pad_start]
        if len(igs) != len(words):
            raise Exception("Length of the gradients and words not matching")
        #select colors for words based on gradients magnitude
        colors = self.colorize(igs)

        #model prediction
        sample_pred = self.model(np.array([sample_sentence,]))
        pred = sample_pred.numpy()       
        
        if not self.display:
            return sample_vector, words, igs.numpy().tolist(), pred[0][0], actual_label
        
        ## Remove padding tokens and plot importance map for actual words only
#         only_words = []
#         only_words_colors=[]
#         for i in range(tf.squeeze(sample_vector_mask).numpy().shape[0]):
#             if tf.squeeze(sample_vector_mask).numpy()[i]==1:
#                 only_words.append(words[i])
#                 only_words_colors.append(colors[i])
        #print the imporatence maps
        print(f"----------------------------------------")
        print(f"input sentence: {sample_sentence}")
        print(f"\nActual Label: {actual_label},  Predicted label: {pred}")
        #print("\n")   
        display(HTML("".join(list(map(self.hlstr, words, colors)))))
        print(f"\n {igs}")
        #return words, colors
    
    def interpolate_texts(self, baseline, text, m_steps):

        """ Linearly interpolate the input vector
        (embedding layer output of the sample vector)"""

        # Generate m_steps intervals for integral_approximation() below.
        alphas = tf.linspace(start=0.0, stop=1.0, num=m_steps+1)
        # text = tf.cast(text, tf.float32)
        alphas_x = alphas[:, tf.newaxis, tf.newaxis]
        delta = text - baseline
        texts = baseline + alphas_x * delta
        return texts
        
    def compute_gradients(self, t, target_class_idx):

        """ compute the gradient wrt to embedding layer output """

        with tf.GradientTape() as tape:
            tape.watch(t)
            probs = self.new_model(t)[:, target_class_idx]
        grads = tape.gradient(probs, t)
        return grads  
    
    def colorize(self, attrs, cmap='PiYG'):
        """
        Compute hex colors based on the attributions for a single instance.
        Uses a diverging colorscale by default and normalizes and scales
        the colormap so that colors are consistent with the attributions.
        """

        cmap_bound = tf.reduce_max(tf.abs(attrs))
        #print(f"cmap_bound:\n{cmap_bound}")
        norm = Normalize(vmin=-cmap_bound, vmax=cmap_bound)
        cmap = mpl.cm.get_cmap(cmap)

        # now compute hex values of colors
        colors = list(map(lambda x: rgb2hex(cmap(norm(x))), attrs))
        return colors    
    
    def  hlstr(self,string, color='white'):
        """
        Return HTML markup highlighting text with the desired color.
        """
        return f"<mark style=background-color:{color};font-size:20px;>{string} </mark>"

In [2]:
### Laod Keras Model

In [3]:
model = tf.keras.models.load_model(r"saved_models\transformer\imdb_keras_base")

In [4]:
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 1)]               0         
                                                                 
 vectorization_layer (TextVe  (None, 200)              0         
 ctorizer)                                                       
                                                                 
 embedding_layer (TokenAndPo  (None, 200, 32)          646400    
 sitionEmbedding)                                                
                                                                 
 tranformer_block1 (Transfor  (None, 200, 32)          10656     
 merBlock)                                                       
                                                                 
 tranformer_block2 (Transfor  (None, 200, 32)          10656     
 merBlock)                                                 

In [11]:
## BERT vocabulary

In [77]:
vocab_file = r'saved_models/transformer/imdb_keras_base_vocab.txt'
# Open the file in read mode
with open(vocab_file, encoding="utf8") as file:
    # Read all the lines of the file into a list
    lines = file.readlines()
    bert_vocab = [line.strip() for line in lines]


In [78]:
bert_vocab[:10]

['', '[UNK]', '[START]', 'the', 'and', 'a', 'of', 'to', 'is', 'in']

### Testign

### Movie reviews

In [125]:
"""
## Download and prepare dataset
"""


dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']
print(f"Length of the training data {len(train_dataset)}")
print(f"Length of the testing data {len(test_dataset)}")

Length of the training data 25000
Length of the testing data 25000


In [126]:
all_results = []
ig = IntegratedGradients(model, 1, 2, bert_vocab, display=False)
for review, label in test_dataset:
    review = review.numpy().decode()
    label = label.numpy()
    result = ig.interpret([review],0,label)
    all_results.append(result)  


In [127]:
len(all_results)

25000

In [129]:
df = pd.DataFrame(data = all_results, columns=['token_ids', 'words', 'igs', 'predicted_label', 'actual_label'])
df.head(2)

,token_ids,words,igs,predicted_label,actual_label
0,"[2, 49, 25, 96, 13, 95, 4156, 17, 727, 5045, 1...","[[START], there, are, films, that, make, caree...","[0.07312308251857758, -0.009299264289438725, -...",0.312348,1
1,"[2, 5, 1, 724, 785, 6, 5, 15046, 2292, 1, 8570...","[[START], a, [UNK], comic, tale, of, a, downtr...","[0.10108847171068192, -0.0005103191360831261, ...",0.546827,1


In [130]:
df.to_csv(r"igs_data_v1.csv",index=False)

## Explanations

In [99]:
example_sentences = [
    ('Adrian Pasdar is excellent in this film. He makes a fascinating woman.',1),
    ('This is the definitive movie version of Hamlet. Branagh cuts nothing, but there are no wasted moments.',1),
    ("I don't know why I like this movie so well, but I never get tired of watching it.",1),    
    ("The film lacked depth, with weak characters and a confusing plot. It failed to engage, leaving me disappointed and uninterested.",0),
    ("The movie's lackluster plot and uninspired performances didn't leave me impressed.",0),
    ("The movie was a complete letdown: weak plot, uninspiring performances, and a total waste of time.",0),
    ("The woman in this movie",0),
    ("The man in this movie",0),
    ("There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.",1),    
]
for sent, label in example_sentences:
    ig.interpret([sent],0, label)
    

----------------------------------------
input sentence: ['Adrian Pasdar is excellent in this film. He makes a fascinating woman.']

Actual Label: 1,  Predicted label: [[0.98911136]]



 [ 1.66434631e-01 -2.48462171e-03 -1.60503834e-02  2.21091621e-02
  3.27799737e-01 -1.36749744e-02 -2.72066519e-03  5.75973559e-03
 -8.21040422e-02  6.79497421e-02  1.09273810e-02  1.65695921e-01
 -7.36741275e-02  4.69050091e-03  4.29429114e-03  2.34020874e-04
 -1.00664971e-02 -7.98670808e-04 -3.19015700e-03  1.12641817e-02
  1.23341708e-03  3.38944700e-03  6.63160160e-03 -2.59839240e-02
  1.13708014e-02 -1.27807790e-02 -2.11773962e-02  5.12036774e-03
 -6.83619874e-03  1.23876547e-02 -3.14372941e-03 -1.18886936e-03
  3.66981328e-03  1.07920617e-02  6.28064666e-03 -6.62999786e-03
  4.72289836e-03 -1.32221123e-03  5.91934472e-03  5.95653430e-03
  4.74296557e-03 -2.51165661e-03  8.15854035e-03 -4.10755258e-03
  1.25119379e-02  1.01299593e-02  1.31452102e-02  3.66613362e-03
  2.80247442e-03 -6.76310016e-03 -6.96125627e-03 -1.20319407e-02
 -1.03391428e-02  1.25063043e-02  2.06736922e-02 -2.59394944e-03
 -4.28170990e-03  7.07576796e-03 -6.75096130e-03 -8.07403959e-03
 -1.93666224e-03  9.143


 [ 4.60300967e-02 -1.04106283e-02  8.84403195e-03  1.34048425e-02
  7.66458139e-02 -1.25000766e-02  1.28937624e-02 -2.97341123e-03
  9.99208167e-03  4.74661402e-02 -6.76601753e-03 -6.54596239e-02
  2.17843987e-03 -3.44785862e-03 -4.99171996e-03 -5.16882166e-02
 -1.44263968e-01  2.26204004e-03 -3.12584476e-03  9.81586240e-03
  2.04854924e-03  2.57192133e-03  6.25096541e-03 -2.34104935e-02
  1.07317157e-02 -1.06300768e-02 -1.84645616e-02  4.80670063e-03
 -5.80090377e-03  1.25485230e-02 -2.72995746e-03 -1.35678111e-03
  2.69059371e-03  9.21112858e-03  4.95491922e-03 -6.26224745e-03
  5.22570731e-03 -9.27028945e-04  5.49018104e-03  5.31085767e-03
  4.07042680e-03 -2.30600918e-03  8.22982565e-03 -3.64706945e-03
  1.10512823e-02  9.84040648e-03  1.11406250e-02  2.46915361e-03
  2.58845882e-03 -6.52008690e-03 -6.12287270e-03 -1.06442329e-02
 -9.36746411e-03  1.15054119e-02  1.83795374e-02 -2.60325987e-03
 -3.45270988e-03  6.91198558e-03 -6.35067187e-03 -7.07935216e-03
 -1.85783044e-03  8.745


 [ 1.71817392e-01 -5.71286771e-04 -5.44223934e-02  3.23808007e-02
 -1.11971691e-01 -2.24713106e-02  2.12970227e-02 -2.92512327e-02
 -7.98016265e-02  2.00828202e-02  2.73068100e-01  2.05234624e-03
  1.32103059e-02  3.51138227e-02  4.41553146e-02 -1.25486016e-01
 -2.62632594e-02 -7.42614418e-02  3.42319980e-02  2.26704199e-02
  3.68045550e-03  6.56978693e-03  1.35836191e-02 -5.48043028e-02
  2.41151229e-02 -2.62118224e-02 -4.34014574e-02  1.04578845e-02
 -1.47314407e-02  2.79542040e-02 -6.80345204e-03 -3.39937746e-03
  6.14203140e-03  2.14153752e-02  1.16647352e-02 -1.55122951e-02
  1.03955446e-02 -2.83376127e-03  1.17222015e-02  1.20101320e-02
  9.22487117e-03 -5.74030261e-03  1.83459446e-02 -8.72820057e-03
  2.50607524e-02  2.10043415e-02  2.58879326e-02  6.09423686e-03
  4.86804638e-03 -1.48471640e-02 -1.39113050e-02 -2.54509840e-02
 -2.22842675e-02  2.60690004e-02  4.24381420e-02 -6.56325929e-03
 -9.41076688e-03  1.48571031e-02 -1.50947897e-02 -1.73107106e-02
 -4.66646906e-03  1.903


 [-1.02270581e-02  6.97889191e-05  6.49452209e-04 -1.08983484e-03
 -4.82852105e-04 -9.45262145e-05 -3.69332545e-03  3.81421414e-05
  3.80024430e-04  6.98146934e-04 -2.75010569e-03 -1.72234734e-03
  8.72801873e-04 -5.07886009e-03  7.10809400e-05 -3.71917849e-05
 -4.15073562e-04  3.58457852e-04 -6.28505182e-03  1.28117937e-03
  7.69221806e-04  1.87969781e-04  3.32463940e-04 -1.02687581e-03
  5.65175200e-04 -4.20782133e-04 -7.92625127e-04  2.98262865e-04
 -2.03224481e-04  6.47930545e-04 -6.60505029e-05 -5.84498775e-06
  1.62729761e-04  4.69250430e-04  2.69234704e-04 -2.56235013e-04
  3.19044571e-04  1.82451913e-05  2.99584412e-04  3.30811774e-04
  2.66899297e-04 -6.79938530e-05  4.23919817e-04 -9.75980947e-05
  6.15394558e-04  5.24771283e-04  5.60466025e-04  1.79585855e-04
  1.84000921e-04 -2.80211680e-04 -2.25718555e-04 -4.48115374e-04
 -3.98878154e-04  6.20661478e-04  9.21517261e-04 -8.34914099e-05
 -1.21422723e-04  3.82338301e-04 -2.13291933e-04 -2.65929091e-04
 -3.71538044e-05  4.541


 [-7.4620442e-03  2.8821587e-04  3.4081088e-03 -1.3840374e-02
 -1.2749240e-02  3.5253661e-03 -3.1899132e-02  1.1722483e-02
 -2.2426851e-02  2.7269500e-03  3.0905479e-03  1.3773781e-02
  8.7410607e-04  9.1450615e-04  1.0101087e-03  2.1135819e-04
 -1.6119755e-03 -8.2023937e-05 -5.5600714e-04  2.1474820e-03
  6.0102751e-04  6.7993603e-04  1.4280786e-03 -4.8582396e-03
  2.4193630e-03 -2.0804137e-03 -3.7448613e-03  1.1386650e-03
 -1.1292445e-03  2.9069842e-03 -4.9078790e-04 -2.3264578e-04
  6.0943142e-04  1.9861890e-03  1.0954936e-03 -1.2779899e-03
  1.3156021e-03 -7.6890574e-05  1.2675565e-03  1.2041053e-03
  9.6743321e-04 -4.7713355e-04  1.9182733e-03 -6.1540457e-04
  2.4657254e-03  2.2530053e-03  2.4281025e-03  6.3969468e-04
  7.0731225e-04 -1.3645398e-03 -1.1795922e-03 -2.1557186e-03
 -1.9057565e-03  2.5771614e-03  3.9919461e-03 -5.1619578e-04
 -6.3369784e-04  1.5840130e-03 -1.1795429e-03 -1.3434775e-03
 -3.4817023e-04  1.9194860e-03 -2.8352113e-04  2.9956785e-04
 -5.0024741e-04  1.651


 [-1.20320870e-02  8.52874364e-05 -4.11192887e-05  1.00201322e-03
  4.37991141e-04 -1.81148457e-03 -5.31939906e-04 -3.10587324e-03
 -3.00762779e-03  6.38069396e-05  4.46502538e-03  8.34031787e-04
  2.61318055e-04 -1.70596177e-03 -1.04810027e-02  1.30516710e-05
  1.89582352e-05  3.95892566e-05 -4.38236893e-05  3.90226312e-04
  1.00330828e-04  1.63329649e-04  2.60900881e-04 -7.38291768e-04
  4.37617535e-04 -2.96330138e-04 -5.62303001e-04  2.36521882e-04
 -1.39864540e-04  5.11059596e-04 -3.12755583e-05  9.51066613e-06
  1.20109995e-04  3.54522112e-04  2.09576974e-04 -1.85756857e-04
  2.53173203e-04  2.53030812e-05  2.28261226e-04  2.59634864e-04
  2.09280843e-04 -4.39406213e-05  3.31354327e-04 -5.56938176e-05
  4.81733790e-04  4.02765087e-04  4.24762926e-04  1.51750486e-04
  1.49924526e-04 -2.07727382e-04 -1.41811135e-04 -3.19474348e-04
 -2.85220332e-04  4.78359056e-04  6.96914853e-04 -4.93018306e-05
 -7.83532596e-05  2.97657680e-04 -1.34611124e-04 -1.67753416e-04
 -1.80765128e-05  3.413


 [ 0.0447471  -0.0013285  -0.22300746  0.03010074 -0.00730678 -0.02433763
  0.01996477 -0.00051307 -0.03412658  0.01219139  0.01759204  0.00223393
  0.006958    0.00715521  0.00739289  0.00047407 -0.01502183 -0.00157491
 -0.00542413  0.01774902  0.00418756  0.00524704  0.01103241 -0.04204707
  0.01930251 -0.01928473 -0.03263164  0.00859188 -0.01091139  0.02299396
 -0.00485334 -0.0023881   0.00479422  0.01663071  0.00891013 -0.0115967
  0.00886429 -0.00133934  0.00925168  0.00939473  0.00764076 -0.00418833
  0.0151622  -0.00639679  0.01970226  0.01737007  0.02001668  0.00500425
  0.0045833  -0.01184193 -0.01046039 -0.0195225  -0.01689103  0.0208149
  0.03302172 -0.00517791 -0.00652511  0.01185016 -0.01089559 -0.01282303
 -0.00335814  0.01517086 -0.0028063   0.00242807 -0.00532047  0.01329274
 -0.01448756 -0.02575505  0.01219375 -0.01563823 -0.00275493 -0.01979022
 -0.0118492   0.02542758  0.01078146 -0.00758243  0.00836037 -0.01135781
  0.01085969  0.00947326  0.01891708 -0.01296371  0


 [ 1.36442482e-01 -5.29642589e-03  3.21952626e-03  3.92650142e-02
 -1.47565845e-02 -3.74251194e-02  2.54232958e-02 -1.58423278e-03
 -4.91907746e-02  1.59529559e-02  2.23836917e-02  3.06558469e-03
  8.83834902e-03  8.94603226e-03  9.66559909e-03 -8.36443156e-04
 -2.07383111e-02 -2.15322152e-03 -7.72847049e-03  2.32058298e-02
  4.15599626e-03  7.44264573e-03  1.37979025e-02 -5.61399236e-02
  2.48693898e-02 -2.66755521e-02 -4.42218706e-02  1.03472658e-02
 -1.53928036e-02  2.91288886e-02 -7.06407148e-03 -3.58937657e-03
  5.93846105e-03  2.15556324e-02  1.18942549e-02 -1.63624529e-02
  9.70342103e-03 -2.82954797e-03  1.15924217e-02  1.21477684e-02
  9.29360185e-03 -6.20044582e-03  1.85261741e-02 -8.61638039e-03
  2.56261528e-02  2.11252142e-02  2.61885133e-02  6.66876603e-03
  4.36408445e-03 -1.52631225e-02 -1.38383042e-02 -2.61445399e-02
 -2.31361948e-02  2.66175158e-02  4.33394462e-02 -6.98645785e-03
 -9.83701088e-03  1.52458716e-02 -1.51399104e-02 -1.73833780e-02
 -4.93125152e-03  1.861


 [ 7.3123083e-02 -9.2992643e-03 -1.2241291e-02  5.9548486e-03
  2.2144872e-03 -4.4769645e-02  1.1272298e-02 -4.5319553e-03
 -4.5360427e-02 -2.9273964e-02  2.6845098e-02 -1.3857229e-02
  7.3723160e-03  1.5877853e-03  6.7304391e-03  2.4472326e-02
 -2.4156721e-02 -6.4879018e-03 -1.0719076e-03  2.3850521e-02
 -1.6683396e-02 -4.2296783e-04  5.8908891e-03 -1.5209125e-02
  1.9449953e-02 -1.9232148e-02 -3.0871466e-02 -3.6478238e-03
 -1.0616899e-02  3.0891733e-02 -9.5614251e-03 -7.8195464e-03
 -3.0002501e-03  1.0090464e-01 -8.6503942e-04  7.2349116e-02
 -2.1606591e-02  1.6163394e-02 -9.4098132e-04  9.4050923e-03
  2.0329865e-02 -9.6983239e-03  2.8381826e-02 -8.7526469e-03
 -1.9332664e-02  8.2575288e-03  1.5270955e-02 -2.3619924e-02
  1.5306981e-02 -4.7206767e-03 -2.7754060e-03  3.1798497e-02
 -1.6433986e-02  5.1948209e-03  3.3100709e-02  2.3097126e-04
 -4.7800224e-04  9.7676860e-03 -1.9428909e-02 -4.8110913e-03
  1.3740256e-02 -2.3548868e-02 -2.3650795e-02  4.8627108e-03
 -2.6653277e-02  4.004